In [1]:
import praw
import numpy as np
import pandas as pd
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
pd.options.mode.chained_assignment = None
import re

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\shash\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Reddit API credentials
client_id = 'Lnt2q1aiGONcbox9EyK-Mw'
client_secret = 'VdTP86mTMJIPop59Xeks_Ldr_AgWCQ'
user_agent = 'test'  # A description of your application

# Initialize PRAW
reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)

In [30]:
def make_clickable(url, name):
    return '<a href="{}" rel="noopener noreferrer" target="_blank">{}</a>'.format(url,name)
def search_reddit(search_word, subreddit_name, search_limit):
    """
    Input: str, str, int
    Output: DataFrame
    """
    # Search for comments related to the specified word
    if subreddit_name:
        subreddit = reddit.subreddit(subreddit_name)
        submissions = subreddit.search(search_word, limit=search_limit)  # You can adjust the limit as needed
    else:
        submissions = reddit.subreddit('all').search(search_word, limit=search_limit)
    authors, comments, urls = [], [], []

    # Iterate through the comments and extract their data
    for submission in submissions:
        authors.append(submission.author.name)
        #print(f"Author: {submission.author}\n")
        comments.append(submission.selftext)
        #print(f"Comment: {submission.selftext}")
        urls.append(submission.url)
    raw_data = pd.DataFrame({'Author':authors, 'Comment':comments, 'URL':urls})
    return raw_data
analyzer = SentimentIntensityAnalyzer()
def score(phrase):
    """
    Input: str
    Output: number between -1 and 1
    """
    return analyzer.polarity_scores(phrase)['compound']
def score_data(data):
    """
    Input: DataFrame
    Output: DataFrame
    """
    new_data = data.copy()
    new_data.drop(axis=0, labels=data.index[data['Comment'].str.len() == 0], inplace=True)
    new_data['Sentiment'] = new_data['Comment'].apply(score)
    #sentiments = new_data[['Author', 'Sentiment']].groupby('Author').mean()
    return new_data#, np.mean(sentiments['Sentiment'])
def format_data(data):
    """
    Input: DataFrame
    Output: Styler
    """
    new_data = data.copy()
    new_data['Comment'] = new_data['Comment'].str[:100]
    new_data['Comment'] = new_data.apply(lambda x: make_clickable(x['URL'], x['Comment']), axis=1)
    return new_data[['Author', 'Comment', 'Sentiment']].style
def score_keyword(search_word, subreddit_name, search_limit):
    raw_data = search_reddit(search_word, subreddit_name, search_limit)
    scored_data = score_data(raw_data)
    return format_data(scored_data)
sentiment_data = score_keyword('anant sahai', 'berkeley', 200)
sentiment_data

,Author,Comment,Sentiment
0,Anon10W1z,(disclaimer that I am a recent graduate and not currently attending) Let's take a look at the [Poli,-0.972100
1,wheezing_cauliflower,​ https://preview.redd.it/yy7cg275sea61.png?width=1348&format=png&auto=webp&s=840b05a6552042,0.000000
2,hotdog_dachshund,As someone taking an easier major on campus (think Haas) I have never fully understood why there are,0.893400
4,eigencar,From eecs101 ​ op: >Directing Concerns/Complaints about enrollment > >It seems like ev,0.993300
5,haehga,I was a student in [CS70](http://inst.eecs.berkeley.edu/%7Ecs70/sp13/) last semester (Spring 2013 wi,-0.995000
6,cs70student2014,[Professor Anant Sahai](http://www.eecs.berkeley.edu/~sahai/) is teaching [CS70 Spring 2014](http://,0.999900
7,RCD-Y,"First and foremost: if this is well-known information, I apologize for wasting your time. But this w",0.983100
8,throwawaythwat,"Also, what does he let you guys call him? Anant Uncle? Mr. Sahai? Dr. Sahai? Professor Sahai? A",0.000000
9,metalreflectslime,Can someone post the full cheating text of the message that Anant Sahai sent out in CS 70? https://,-0.557400
10,cs70student2014,Previous CS70 Spring 2014 cheating thread: http://www.reddit.com/r/berkeley/comments/2401hc/cs70_sel,-0.981500


In [10]:
sentiment_data.sort_values('Sentiment').iloc[-3:, 1].tolist()

["EECS 101 Piazza post: [https://piazza.com/class/hyq0br1u3kx7dg?cid=3467](https://piazza.com/class/hyq0br1u3kx7dg?cid=3467)\n\n## **Student's Piazza post**\n\n> **What is the ultimate goal of our CS classes? How are they designed?**\n> \n> I was talking to an alum who graduated in the early 2000s, and he was talking about being a good software engineer vs. computer scientist.\n> \n> I don't fully understand the distinction between the two. Can someone explain it, and what the school classes are geared towards?\n> \n> What are the professors' expectations when teaching? Do they simply want us to become better at the topic, or do they want us to become a better X through taking courses Y and Z?\n> \n> What does the school want us to become/earn after we graduate besides the degree itself?\n\n## **Anant Sahai's Answer**\n\n[EECS Professor Anant Sahai](https://www2.eecs.berkeley.edu/Faculty/Homepages/sahai.html) replied:\n\n> The point of the classes that EECS offers is to, collectively, 

In [4]:
### Everything below here is unused


def process_data(data):
    new_data = data.copy()
    comments = new_data['Comment']
    comments = comments.str.lower()
    comments = comments.str.replace(r'http.*($|\s)', '') # Get rid of images
    comments = comments.str.replace(r'[^\w|_|\s]|\n', ' ') # Get rid of punctuation and newlines
    new_data['Split'] = comments.str.split() # Create split column
    new_data = new_data[new_data['Split'].str.len() > 1] # Remove empty or one word rows
    new_data = new_data.set_index('Author')
    return new_data
def words_only(data):
    words = data["Split"].explode().to_frame().rename(columns={"Split": "word"})
    return words
def add_polarity(data, words, sentiment):
    new_data = data.copy()
    new_data["Polarity"] = (
        words
        .merge(sentiment, how='left', left_on='word', right_index=True)
        .reset_index()[['Author', 'polarity']]
        .dropna()
        .groupby('Author')
        .mean()
    )
    return new_data

raw_data = search_reddit('anant sahai', 'berkeley', 1000)
processed = process_data(raw_data)
with_polarity = add_polarity(processed, words_only(processed), vader_txt)
np.mean(with_polarity['Polarity'])

/var/folders/73/y83rvk7n519_l2zy_6g46f_h0000gn/T/ipykernel_18009/3206816101.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  comments = comments.str.replace(r'http.*($|\s)', '') # Get rid of images
/var/folders/73/y83rvk7n519_l2zy_6g46f_h0000gn/T/ipykernel_18009/3206816101.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  comments = comments.str.replace(r'[^\w|_|\s]|\n', ' ') # Get rid of punctuation and newlines


NameError: name 'vader_txt' is not defined

In [23]:
vader_txt = pd.read_csv('vader_lexicon.txt', delimiter = '\t', names = ['token', 'polarity', 'x', 'y'])[['token', 'polarity']].set_index('token')
vader_txt

,polarity
token,
$:,-1.5
%),-0.4
%-),-1.5
&-:,-0.4
&:,-0.7
...,...
}:,-2.1
}:(,-2.0
}:),0.4


In [338]:
# Define the word you want to search for
search_word = 'cs 70 rao'

# Specify the subreddit where you want to search for comments (optional)
# If you want to search all of Reddit, set subreddit_name to None
subreddit_name = 'berkeley'  # Change this to the subreddit you're interested in

In [310]:
# Limit of entries to search for
search_limit = 1000

# Search for comments related to the specified word
if subreddit_name:
    subreddit = reddit.subreddit(subreddit_name)
    submissions = subreddit.search(search_word, limit=search_limit)  # You can adjust the limit as needed
else:
    submissions = reddit.subreddit('all').search(search_word, limit=search_limit)

In [311]:
# Initialize lists to store authors and comments
authors, comments, urls = [], [], []

# Iterate through the comments and extract their data
for submission in submissions:
    authors.append(submission.author.name)
    #print(f"Author: {submission.author}\n")
    comments.append(submission.selftext)
    #print(f"Comment: {submission.selftext}")
    urls.append(submission.url)
raw_data = pd.DataFrame({'Author':authors, 'Comment':comments, 'URL':urls})

In [312]:
raw_data

,Author,Comment,URL
0,YogurtMan05,LSCS Freshman looking for advice on whether or...,https://www.reddit.com/r/berkeley/comments/175...
1,sakenikuyee,How is CS 70 with Professor Rao? Rate my profe...,https://www.reddit.com/r/berkeley/comments/jip...
2,genius_gates,Since grades have come out and an influx of po...,https://www.reddit.com/r/berkeley/comments/ndh...
3,Ok_Comedian3248,How hard is CS 170 w/o having taken CS 70? \n\...,https://www.reddit.com/r/berkeley/comments/174...
4,exclusiveassmofo,Whoever made this years midterm needs to be fi...,https://www.reddit.com/r/berkeley/comments/11m...
...,...,...,...
219,tums_festival47,I’m finding DATA 8 pretty easy and 61a so far ...,https://www.reddit.com/r/berkeley/comments/j2p...
220,indecibel_moe,Hi /r/berkeley!\n\nI'm an incoming freshman ta...,https://www.reddit.com/r/berkeley/comments/c67...
221,capitan_presidente,"Hi everybody, \n\nI wanted to see if anybody w...",https://www.reddit.com/r/berkeley/comments/ify...
222,Assignment_Relative,,https://i.redd.it/ui4kdzvulk5a1.jpg


In [315]:
# anthro2ac: 0.065
# cs61a: 0.020
# cs61b: 0.046
# cs70: 0.016
# math110: 0.018
# math54: 0.056